Arjun Bhan (AB5666) and Leah Uzzan (lu2166)

In [ ]:
# Function to load a single batch of CIFAR-10 data
def load_cifar10_batch(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
# Load CIFAR-10 data
data_folder = '/kaggle/input/Cifar10-ADL/cifar-10-batches-py/'  # Replace with your path
batch_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']
#data_batches = [load_cifar10_batch(os.path.join(data_folder, f'{file}')) for file in batch_files]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [ ]:
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import StepLR

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Data augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset with augmentation
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Splitting full training dataset into training and validation sets
train_size = int(0.8 * len(full_train_dataset))
validation_size = len(full_train_dataset) - train_size
train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, validation_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:03<00:00, 43228490.08it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Traditional VGGNet

In [ ]:
class VGG16Features(nn.Module):
    def __init__(self):
        super(VGG16Features, self).__init__()
        self.features = nn.Sequential(
            # First block
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Second block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Third block
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Fourth block
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Fifth block
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

    def forward(self, x):
        x = self.features(x)
        return x


In [ ]:
class VGG16Classifier(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16Classifier, self).__init__()
        # The input feature size is 512 (from the conv layers) * 1 * 1
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.classifier(x)
        return x

In [ ]:
class ModifiedVGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedVGG16, self).__init__()
        self.features = VGG16Features()
        input_channels = 512  # Output channels of last conv layer in VGG16Features
        self.classifier = VGG16Classifier(num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        output = self.classifier(x)
        return output

In [ ]:
from tqdm import tqdm

# Function to train and validate the model
def train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        train_correct = 0
        train_loss = 0.0
        train_total = 0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()

        train_loss /= train_total
        train_accuracy = 100 * train_correct / train_total

        # Validation loop
        model.eval()
        val_correct = 0
        val_loss = 0.0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        val_loss /= val_total
        val_accuracy = 100 * val_correct / val_total

        print(f'Epoch {epoch+1}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

        scheduler.step(val_loss)

In [ ]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')


In [ ]:
# Model, Loss Function, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedVGG16(num_classes=10).to(device)
# Use AdamW optimizer and StepLR scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
# Use a learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

# Loss function?
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train the model
train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs=20)

100%|██████████| 625/625 [00:27<00:00, 22.45it/s]


Epoch 1, Training Loss: 2.0716, Training Accuracy: 17.53%, Validation Loss: 1.9309, Validation Accuracy: 23.60%


100%|██████████| 625/625 [00:27<00:00, 23.14it/s]


Epoch 2, Training Loss: 1.7874, Training Accuracy: 29.07%, Validation Loss: 1.6865, Validation Accuracy: 35.51%


100%|██████████| 625/625 [00:26<00:00, 23.66it/s]


...

Epoch 18, Training Loss: 0.4608, Training Accuracy: 84.48%, Validation Loss: 0.6579, Validation Accuracy: 78.47%


100%|██████████| 625/625 [00:27<00:00, 22.61it/s]


Epoch 19, Training Loss: 0.4359, Training Accuracy: 85.41%, Validation Loss: 0.6679, Validation Accuracy: 78.38%


100%|██████████| 625/625 [00:26<00:00, 23.35it/s]


Epoch 20, Training Loss: 0.4098, Training Accuracy: 86.25%, Validation Loss: 0.6872, Validation Accuracy: 78.67%


In [ ]:
# Training and Evaluation
evaluate_model(model, test_loader)

Accuracy: 80.02%


VGGNet with PNet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PNet(nn.Module):
    def __init__(self, input_channels, attention_channels):
        super(PNet, self).__init__()
        # Assuming the input feature map has a shape of [batch_size, input_channels, height, width]
        # Convolutional layers to process feature maps
        self.conv1 = nn.Conv2d(input_channels, attention_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(attention_channels, attention_channels, kernel_size=3, padding=1)
        # Output layer to generate attention map with a single channel
        self.output_layer = nn.Conv2d(attention_channels, 1, kernel_size=3, padding=1)

    def forward(self, x):
        # Pass input through convolutional layers with a non-linear activation function (e.g., ReLU)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        # Generate the attention map
        # We use a sigmoid function to ensure the output values are between 0 and 1
        attention_map = torch.sigmoid(self.output_layer(x))
        return attention_map


In [ ]:
class VGG16Features(nn.Module):
    def __init__(self):
        super(VGG16Features, self).__init__()
        self.features = nn.Sequential(
            # First block
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Second block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Third block
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Fourth block
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Fifth block
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

    def forward(self, x):
        x = self.features(x)
        return x


In [ ]:
class VGG16Classifier(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16Classifier, self).__init__()
        # The input feature size is 512 (from the conv layers) * 1 * 1
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.classifier(x)
        return x

In [ ]:
class ModifiedVGG16(nn.Module):
    def __init__(self, num_classes=10, attention_channels=64):
        super(ModifiedVGG16, self).__init__()
        self.features = VGG16Features()
        input_channels = 512  # Output channels of last conv layer in VGG16Features
        self.pnet = PNet(input_channels, attention_channels)
        self.classifier = VGG16Classifier(num_classes)

    def forward(self, x):
        features = self.features(x)
        attention_maps = self.pnet(features)
        attended_features = features * attention_maps

        # Flatten the attended features to match the classifier's input size
        attended_features = attended_features.view(attended_features.size(0), -1)

        output = self.classifier(attended_features)
        return output

In [ ]:
from tqdm import tqdm

# Function to train and validate the model
def train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        train_correct = 0
        train_loss = 0.0
        train_total = 0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()

        train_loss /= train_total
        train_accuracy = 100 * train_correct / train_total

        # Validation loop
        model.eval()
        val_correct = 0
        val_loss = 0.0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        val_loss /= val_total
        val_accuracy = 100 * val_correct / val_total

        print(f'Epoch {epoch+1}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

        scheduler.step(val_loss)

In [ ]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')


In [ ]:
# Model, Loss Function, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedVGG16(num_classes=10).to(device)
# Use AdamW optimizer and StepLR scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
# Use a learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

# Loss function?
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train the model
train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs=20)

100%|██████████| 625/625 [00:28<00:00, 21.80it/s]


Epoch 1, Training Loss: 2.1158, Training Accuracy: 16.39%, Validation Loss: 1.8863, Validation Accuracy: 24.55%


100%|██████████| 625/625 [00:26<00:00, 23.42it/s]


Epoch 2, Training Loss: 1.7879, Training Accuracy: 29.90%, Validation Loss: 1.6566, Validation Accuracy: 36.72%


100%|██████████| 625/625 [00:27<00:00, 23.13it/s]


...

Epoch 18, Training Loss: 0.4681, Training Accuracy: 83.90%, Validation Loss: 0.6621, Validation Accuracy: 78.57%


100%|██████████| 625/625 [00:26<00:00, 23.38it/s]


Epoch 19, Training Loss: 0.4367, Training Accuracy: 85.17%, Validation Loss: 0.6559, Validation Accuracy: 78.76%


100%|██████████| 625/625 [00:26<00:00, 23.74it/s]


Epoch 20, Training Loss: 0.4100, Training Accuracy: 86.11%, Validation Loss: 0.7205, Validation Accuracy: 76.90%


In [ ]:
# Training and Evaluation
evaluate_model(model, test_loader)

Accuracy: 78.53%


VGGNet with PNet and updates

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PNet(nn.Module):
    def __init__(self, input_channels, attention_channels):
        super(PNet, self).__init__()
        # Assuming the input feature map has a shape of [batch_size, input_channels, height, width]
        # Convolutional layers to process feature maps
        self.conv1 = nn.Conv2d(input_channels, attention_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(attention_channels, attention_channels, kernel_size=3, padding=1)
        # Output layer to generate attention map with a single channel
        self.output_layer = nn.Conv2d(attention_channels, 1, kernel_size=3, padding=1)

    def forward(self, x):
        # Pass input through convolutional layers with a non-linear activation function (e.g., ReLU)
        x = F.leaky_relu(self.conv1(x), negative_slope = 0.01)
        x = F.leaky_relu(self.conv2(x), negative_slope = 0.01)
        # Generate the attention map
        # We use a sigmoid function to ensure the output values are between 0 and 1
        attention_map = torch.sigmoid(self.output_layer(x))
        return attention_map


In [ ]:
class VGG16Features(nn.Module):
    def __init__(self):
        super(VGG16Features, self).__init__()
        self.features = nn.Sequential(
            # First block
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3),

            # Second block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3),

            # Third block
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3),

            # Fourth block
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3),

            # Fifth block
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3),
        )

    def forward(self, x):
        x = self.features(x)
        return x


In [ ]:
class VGG16Classifier(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16Classifier, self).__init__()
        # The input feature size is 512 (from the conv layers) * 1 * 1
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.classifier(x)
        return x

In [ ]:
class ModifiedVGG16(nn.Module):
    def __init__(self, num_classes=10, attention_channels=64):
        super(ModifiedVGG16, self).__init__()
        self.features = VGG16Features()
        input_channels = 512  # Output channels of last conv layer in VGG16Features
        self.pnet = PNet(input_channels, attention_channels)
        self.classifier = VGG16Classifier(num_classes)

    def forward(self, x):
        features = self.features(x)
        attention_maps = self.pnet(features)
        attended_features = features * attention_maps

        # Flatten the attended features to match the classifier's input size
        attended_features = attended_features.view(attended_features.size(0), -1)

        output = self.classifier(attended_features)
        return output

In [ ]:
from tqdm import tqdm

# Function to train and validate the model
def train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        train_correct = 0
        train_loss = 0.0
        train_total = 0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()

        train_loss /= train_total
        train_accuracy = 100 * train_correct / train_total

        # Validation loop
        model.eval()
        val_correct = 0
        val_loss = 0.0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        val_loss /= val_total
        val_accuracy = 100 * val_correct / val_total

        print(f'Epoch {epoch+1}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

        scheduler.step(val_loss)

In [ ]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')


In [ ]:
# Model, Loss Function, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedVGG16(num_classes=10).to(device)
# Use AdamW optimizer and StepLR scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
# Use a learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

# Loss function?
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train the model
train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs=20)

100%|██████████| 625/625 [00:28<00:00, 22.09it/s]


Epoch 1, Training Loss: 1.6963, Training Accuracy: 34.71%, Validation Loss: 1.5421, Validation Accuracy: 45.56%


100%|██████████| 625/625 [00:28<00:00, 22.07it/s]


Epoch 2, Training Loss: 1.2788, Training Accuracy: 54.02%, Validation Loss: 1.6311, Validation Accuracy: 48.18%


100%|██████████| 625/625 [00:28<00:00, 22.31it/s]


...

Epoch 18, Training Loss: 0.4723, Training Accuracy: 83.91%, Validation Loss: 0.5241, Validation Accuracy: 82.09%


100%|██████████| 625/625 [00:28<00:00, 21.78it/s]


Epoch 19, Training Loss: 0.4597, Training Accuracy: 84.43%, Validation Loss: 0.4939, Validation Accuracy: 83.81%


100%|██████████| 625/625 [00:28<00:00, 21.79it/s]


Epoch 20, Training Loss: 0.4378, Training Accuracy: 85.15%, Validation Loss: 0.4953, Validation Accuracy: 82.63%


In [ ]:
# Training and Evaluation
evaluate_model(model, test_loader)

Accuracy: 84.43%


Traditional CNN

In [ ]:
class TradCNN(nn.Module):

  def __init__(self):
    super().__init__()
    self.cnn1 = nn.Conv2d(3, 30, kernel_size= 3, padding=1)
    self.cnn2 = nn.Conv2d(30, 60, kernel_size= 3, padding=1)
    self.cnn3 = nn.Conv2d(60, 90, kernel_size= 3, padding=1)
    self.Linear1 = nn.Linear(90 * 4 *4, 480)
    self.Linear2 = nn.Linear(480, 10)
    self.maxPool = nn.MaxPool2d(kernel_size= 2, stride=2)
    self.relu = nn.ReLU()


  def forward(self, x):
    x = self.cnn1(x)
    x = self.relu(x)
    x = self.maxPool(x)

    x = self.cnn2(x)
    x = self.relu(x)
    x = self.maxPool(x)

    x = self.cnn3(x)
    x = self.relu(x)
    x = self.maxPool(x)

    x = torch.flatten(x, start_dim = 1)

    x = self.Linear1(x)
    x = self.relu(x)
    x = self.Linear2(x)
    return x


In [ ]:
from tqdm import tqdm

# Function to train and validate the model
def train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        train_correct = 0
        train_loss = 0.0
        train_total = 0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()

        train_loss /= train_total
        train_accuracy = 100 * train_correct / train_total

        # Validation loop
        model.eval()
        val_correct = 0
        val_loss = 0.0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        val_loss /= val_total

        print(f'Epoch {epoch+1}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

        scheduler.step(val_loss)

In [ ]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')


In [ ]:
# Model, Loss Function, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TradCNN().to(device)
# Use AdamW optimizer and StepLR scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
# Use a learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

# Loss function?
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train the model
train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs=20)

100%|██████████| 625/625 [00:20<00:00, 31.04it/s]


Epoch 1, Training Loss: 1.8036, Training Accuracy: 35.24%, Validation Loss: 1.5287, Validation Accuracy: 44.82%


100%|██████████| 625/625 [00:21<00:00, 29.68it/s]


Epoch 2, Training Loss: 1.4722, Training Accuracy: 46.84%, Validation Loss: 1.4209, Validation Accuracy: 48.59%


100%|██████████| 625/625 [00:20<00:00, 30.65it/s]


...

Epoch 18, Training Loss: 0.8701, Training Accuracy: 69.67%, Validation Loss: 0.9535, Validation Accuracy: 67.01%


100%|██████████| 625/625 [00:20<00:00, 30.52it/s]


Epoch 19, Training Loss: 0.8511, Training Accuracy: 70.51%, Validation Loss: 0.9036, Validation Accuracy: 68.13%


100%|██████████| 625/625 [00:20<00:00, 30.61it/s]


Epoch 20, Training Loss: 0.8380, Training Accuracy: 70.97%, Validation Loss: 0.8863, Validation Accuracy: 68.75%


In [ ]:
# Training and Evaluation
evaluate_model(model, test_loader)

Accuracy: 69.72%
